In [1]:
import pandas as pd

parent_dir = "../data/conceptual-error-candidates/validation_catalog_"

df_ali = pd.read_csv(parent_dir + "ali.csv")
df_arvind = pd.read_csv(parent_dir + "arvind.csv")
df_ling = pd.read_csv(parent_dir + "ling.csv")
df_mauro = pd.read_csv(parent_dir + "mauro.csv")
df_yewei = pd.read_csv(parent_dir + "yewei.csv")

df_ali = df_ali[df_ali["status"] == "accepted"]
df_arvind = df_arvind[df_arvind["status"] == "accepted"]
df_ling = df_ling[df_ling["status"] == "accepted"]
df_mauro = df_mauro[df_mauro["status"] == "accepted"]
df_yewei = df_yewei[df_yewei["status"] == "accepted"]

print("Ali:", len(df_ali))
print("Arvind:", len(df_arvind))
print("Ling:", len(df_ling))
print("Mauro:", len(df_mauro))
print("Yewei:", len(df_yewei))
print("Total:", len(df_ali) + len(df_arvind) + len(df_ling) + len(df_mauro) + len(df_yewei))

Ali: 341
Arvind: 91
Ling: 110
Mauro: 312
Yewei: 290
Total: 1144


In [2]:
validation_catalogs = {
    "ali": df_ali,
    "arvind": df_arvind,
    "ling": df_ling,
    "mauro": df_mauro,
    "yewei": df_yewei
}

In [3]:
import pandas as pd
import os
from pathlib import Path

# Function to check if a file exists in the accepted folder
def check_file_exists_in_accepted(filepath, base_dir="../data/conceptual-errors-accepted/"):
    """
    Check if a file exists in the conceptual-errors-accepted folder.
    The filepath in the catalog points to the accepted location if accepted.
    """
    if pd.isna(filepath) or filepath == "":
        return False
    
    # Convert to Path object and normalize path separators
    # Replace Windows backslashes with forward slashes for cross-platform compatibility
    normalized_filepath = str(filepath).replace('\\', '/')
    file_path = Path(normalized_filepath)
    
    # If it's a relative path, make it relative to the notebook location
    if not file_path.is_absolute():
        # Assuming we're running from notebooks/ directory
        full_path = Path("..") / file_path
    else:
        full_path = file_path
    
    return full_path.exists()

# Main diagnostic function
def run_diagnostics(validation_catalogs):
    """
    Run diagnostic checks on validation catalogs vs actual files in accepted folder.
    """
    print("=" * 80)
    print("DIAGNOSTIC REPORT: Validation Catalog vs Accepted Files")
    print("=" * 80)
    
    total_inconsistencies = 0
    
    for reviewer_name, df in validation_catalogs.items():
        print(f"\n📋 REVIEWER: {reviewer_name.upper()}")
        print("-" * 50)
        
        # Create a copy to avoid modifying original
        df_check = df.copy()
        
        # Add the 'added' column by checking if files exist
        df_check['added'] = df_check['filepath'].apply(check_file_exists_in_accepted)
        
        # Create cross-tabulation
        crosstab = pd.crosstab(df_check['status'], df_check['added'], 
                              margins=True, margins_name='Total')
        
        print("Cross-tabulation (Status vs File Actually Added):")
        print(crosstab)
        print()
        
        # Identify specific inconsistencies
        accepted_but_not_added = df_check[(df_check['status'] == 'accepted') & 
                                         (df_check['added'] == False)]
        not_accepted_but_added = df_check[(df_check['status'] != 'accepted') & 
                                         (df_check['added'] == True)]
        
        inconsistency_count = len(accepted_but_not_added) + len(not_accepted_but_added)
        total_inconsistencies += inconsistency_count
        
        if inconsistency_count > 0:
            print("🚨 INCONSISTENCIES FOUND:")
            
            if len(accepted_but_not_added) > 0:
                print(f"   • {len(accepted_but_not_added)} samples marked 'accepted' but file NOT found:")
                for idx, row in accepted_but_not_added.iterrows():
                    print(f"     - Index {row['index']}: {row['tier']} | {row['mutation_type']} | {row['target_variable']}")
                    print(f"       Expected file: {row['filepath']}")
                    # Show normalized path for debugging
                    normalized_path = str(row['filepath']).replace('\\', '/')
                    full_check_path = Path("..") / normalized_path
                    print(f"       Checking path: {full_check_path}")
                    print(f"       Path exists: {full_check_path.exists()}")
                print()
            
            if len(not_accepted_but_added) > 0:
                print(f"   • {len(not_accepted_but_added)} samples NOT marked 'accepted' but file exists:")
                for idx, row in not_accepted_but_added.iterrows():
                    print(f"     - Index {row['index']}: Status='{row['status']}' but file exists at {row['filepath']}")
                print()
        else:
            print("✅ No inconsistencies found!")
        
        # Summary statistics
        total_samples = len(df_check)
        accepted_count = len(df_check[df_check['status'] == 'accepted'])
        files_found = len(df_check[df_check['added'] == True])
        
        print(f"📊 SUMMARY:")
        print(f"   • Total samples in catalog: {total_samples}")
        print(f"   • Samples marked 'accepted': {accepted_count}")
        print(f"   • Files actually found: {files_found}")
        print(f"   • Inconsistencies: {inconsistency_count}")
        
        # Calculate percentages
        if accepted_count > 0:
            success_rate = ((accepted_count - len(accepted_but_not_added)) / accepted_count) * 100
            print(f"   • Success rate (accepted → file created): {success_rate:.1f}%")
        
    print("\n" + "=" * 80)
    print(f"🎯 OVERALL SUMMARY:")
    print(f"   • Total reviewers checked: {len(validation_catalogs)}")
    print(f"   • Total inconsistencies across all reviewers: {total_inconsistencies}")
    if total_inconsistencies == 0:
        print("   • 🎉 All validation catalogs are consistent with accepted files!")
    else:
        print(f"   • ⚠️  Found {total_inconsistencies} inconsistencies that need investigation.")
    print("=" * 80)

# Run the diagnostics
run_diagnostics(validation_catalogs)

DIAGNOSTIC REPORT: Validation Catalog vs Accepted Files

📋 REVIEWER: ALI
--------------------------------------------------
Cross-tabulation (Status vs File Actually Added):
added     True  Total
status               
accepted   341    341
Total      341    341

✅ No inconsistencies found!
📊 SUMMARY:
   • Total samples in catalog: 341
   • Samples marked 'accepted': 341
   • Files actually found: 341
   • Inconsistencies: 0
   • Success rate (accepted → file created): 100.0%

📋 REVIEWER: ARVIND
--------------------------------------------------
Cross-tabulation (Status vs File Actually Added):
added     True  Total
status               
accepted    91     91
Total       91     91

✅ No inconsistencies found!
📊 SUMMARY:
   • Total samples in catalog: 91
   • Samples marked 'accepted': 91
   • Files actually found: 91
   • Inconsistencies: 0
   • Success rate (accepted → file created): 100.0%

📋 REVIEWER: LING
--------------------------------------------------
Cross-tabulation (Status vs

In [ ]:
import pandas as pd
import json
from pathlib import Path

# Load Ling's copied catalog
def load_ling_copy_catalog():
    """Load Ling's copied validation catalog"""
    ling_copy_path = "../data/conceptual-error-candidates/validation_catalog_ling_copy.csv"
    df_ling_copy = pd.read_csv(ling_copy_path)
    return df_ling_copy

# Function to check if a file exists in Ling's copied accepted folder
def check_file_exists_in_ling_copy(filepath, base_dir="../data/conceptual-errors-accepted-copy/"):
    """
    Check if a file exists in Ling's copied conceptual-errors-accepted folder.
    """
    if pd.isna(filepath) or filepath == "":
        return False
    
    # Convert to Path object and normalize path separators
    normalized_filepath = str(filepath).replace('\\', '/')
    file_path = Path(normalized_filepath)
    
    # Extract the relative path from conceptual-errors-accepted onwards
    # Handle both cases: full path or relative path
    if "conceptual-errors-accepted" in str(file_path):
        # Extract everything after "conceptual-errors-accepted"
        parts = file_path.parts
        try:
            accepted_idx = next(i for i, part in enumerate(parts) if "conceptual-errors-accepted" in part)
            relative_path = Path(*parts[accepted_idx + 1:])
        except StopIteration:
            # If we can't find the pattern, use the full path as is
            relative_path = file_path
    else:
        relative_path = file_path
    
    # Build the full path to Ling's copy
    if not relative_path.is_absolute():
        full_path = Path("..") / "data" / "conceptual-errors-accepted-copy" / relative_path
    else:
        full_path = relative_path
    
    return full_path.exists()

# Main diagnostic function for Ling's copy
def run_ling_copy_diagnostics():
    """
    Run diagnostic checks on Ling's copied validation catalog vs copied accepted files.
    """
    print("=" * 80)
    print("DIAGNOSTIC REPORT: Ling's Copied Files")
    print("=" * 80)
    
    # Load Ling's copied catalog
    df_ling_copy = load_ling_copy_catalog()
    
    print(f"📋 LING'S COPIED CATALOG")
    print("-" * 50)
    print(f"Total entries in catalog: {len(df_ling_copy)}")
    
    # Show status distribution
    status_counts = df_ling_copy['status'].value_counts()
    print("Status distribution:")
    for status, count in status_counts.items():
        print(f"   • {status}: {count}")
    print()
    
    # Filter for accepted entries only
    df_accepted = df_ling_copy[df_ling_copy['status'] == 'accepted'].copy()
    print(f"Accepted entries: {len(df_accepted)}")
    
    if len(df_accepted) == 0:
        print("No accepted entries found in Ling's catalog.")
        return
    
    # Add the 'added' column by checking if files exist in the copy folder
    df_accepted['added'] = df_accepted['filepath'].apply(check_file_exists_in_ling_copy)
    
    # Create cross-tabulation
    crosstab = pd.crosstab(df_accepted['status'], df_accepted['added'], 
                          margins=True, margins_name='Total')
    
    print("Cross-tabulation (Status vs File Actually Added in Copy):")
    print(crosstab)
    print()
    
    # Identify specific inconsistencies
    accepted_but_not_added = df_accepted[(df_accepted['status'] == 'accepted') & 
                                       (df_accepted['added'] == False)]
    
    inconsistency_count = len(accepted_but_not_added)
    
    if inconsistency_count > 0:
        print("🚨 INCONSISTENCIES FOUND:")
        print(f"   • {len(accepted_but_not_added)} samples marked 'accepted' but file NOT found in copy:")
        
        for idx, row in accepted_but_not_added.iterrows():
            print(f"     - Index {row['index']}: {row['tier']} | {row['mutation_type']} | {row['target_variable']}")
            print(f"       Expected file: {row['filepath']}")
            
            # Show what path we're actually checking
            normalized_filepath = str(row['filepath']).replace('\\', '/')
            file_path = Path(normalized_filepath)
            
            if "conceptual-errors-accepted" in str(file_path):
                parts = file_path.parts
                try:
                    accepted_idx = next(i for i, part in enumerate(parts) if "conceptual-errors-accepted" in part)
                    relative_path = Path(*parts[accepted_idx + 1:])
                except StopIteration:
                    relative_path = file_path
            else:
                relative_path = file_path
                
            full_check_path = Path("..") / "data" / "conceptual-errors-accepted-copy" / relative_path
            print(f"       Checking path: {full_check_path}")
            print(f"       Path exists: {full_check_path.exists()}")
        print()
    else:
        print("✅ No inconsistencies found!")
    
    # Summary statistics
    total_accepted = len(df_accepted)
    files_found = len(df_accepted[df_accepted['added'] == True])
    
    print(f"📊 SUMMARY:")
    print(f"   • Total accepted samples in catalog: {total_accepted}")
    print(f"   • Files actually found in copy folder: {files_found}")
    print(f"   • Inconsistencies: {inconsistency_count}")
    
    # Calculate success rate
    if total_accepted > 0:
        success_rate = (files_found / total_accepted) * 100
        print(f"   • Success rate (accepted → file found in copy): {success_rate:.1f}%")
    
    print("=" * 80)
    
    return df_accepted

# Load and analyze Ling's copied files
print("Loading Ling's copied catalog...")
df_ling_copy_analysis = run_ling_copy_diagnostics()

Loading Ling's copied catalog...
DIAGNOSTIC REPORT: Ling's Copied Files
📋 LING'S COPIED CATALOG
--------------------------------------------------
Total entries in catalog: 388
Status distribution:
   • todo: 187
   • accepted: 110
   • rejected: 87
   • skipped: 4

Accepted entries: 110
Cross-tabulation (Status vs File Actually Added in Copy):
added     True  Total
status               
accepted   110    110
Total      110    110

✅ No inconsistencies found!
📊 SUMMARY:
   • Total accepted samples in catalog: 110
   • Files actually found in copy folder: 110
   • Inconsistencies: 0
   • Success rate (accepted → file found in copy): 100.0%


In [8]:
import pandas as pd
import json
import shutil
from pathlib import Path

def move_ling_files_to_main():
    """
    Move all of Ling's accepted files from the copy folder to the main accepted folder
    and update Ling's validation catalog.
    """
    print("=" * 80)
    print("MOVING LING'S FILES TO MAIN FOLDERS")
    print("=" * 80)
    
    # Paths
    copy_folder = Path("../data/conceptual-errors-accepted-copy")
    main_folder = Path("../data/conceptual-errors-accepted")
    copy_catalog_path = Path("../data/conceptual-error-candidates/validation_catalog_ling_copy.csv")
    main_catalog_path = Path("../data/conceptual-error-candidates/validation_catalog_ling.csv")
    
    # Step 1: Load the copy catalog to see what we're working with
    print("📋 Loading Ling's copy catalog...")
    df_ling_copy = pd.read_csv(copy_catalog_path)
    accepted_entries = df_ling_copy[df_ling_copy['status'] == 'accepted']
    
    print(f"Found {len(accepted_entries)} accepted entries to process")
    print()
    
    # Step 2: Move accepted files
    print("📁 Moving accepted files...")
    moved_count = 0
    error_count = 0
    overwritten_count = 0
    
    for idx, row in accepted_entries.iterrows():
        try:
            # Parse the filepath to get the relative path structure
            filepath = str(row['filepath']).replace('\\', '/')
            file_path = Path(filepath)
            
            # Extract relative path from conceptual-errors-accepted onwards
            if "conceptual-errors-accepted" in str(file_path):
                parts = file_path.parts
                try:
                    accepted_idx = next(i for i, part in enumerate(parts) if "conceptual-errors-accepted" in part)
                    relative_path = Path(*parts[accepted_idx + 1:])
                except StopIteration:
                    relative_path = file_path
            else:
                relative_path = file_path
            
            # Source and destination paths
            source_path = copy_folder / relative_path
            dest_path = main_folder / relative_path
            
            # Check if source exists
            if not source_path.exists():
                print(f"❌ Source file not found: {source_path}")
                error_count += 1
                continue
            
            # Create destination directory if it doesn't exist
            dest_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Check if destination already exists
            will_overwrite = dest_path.exists()
            if will_overwrite:
                overwritten_count += 1
                print(f"⚠️  Overwriting: {relative_path}")
            else:
                print(f"✅ Moving: {relative_path}")
            
            # Copy the file (this will overwrite if destination exists)
            shutil.copy2(source_path, dest_path)
            moved_count += 1
            
        except Exception as e:
            print(f"❌ Error processing {row['index']}: {e}")
            error_count += 1
    
    print()
    print(f"📊 File Movement Summary:")
    print(f"   • Successfully moved: {moved_count}")
    print(f"   • Overwritten existing files: {overwritten_count}")
    print(f"   • Errors: {error_count}")
    print()
    
    # Step 3: Replace the main catalog with the copy
    print("📋 Updating Ling's main validation catalog...")
    
    try:
        # Create backup of original catalog if it exists
        if main_catalog_path.exists():
            backup_path = main_catalog_path.with_suffix('.csv.backup')
            shutil.copy2(main_catalog_path, backup_path)
            print(f"✅ Created backup: {backup_path}")
        
        # Copy the catalog
        shutil.copy2(copy_catalog_path, main_catalog_path)
        print(f"✅ Updated main catalog: {main_catalog_path}")
        
        # Verify the update
        df_new_main = pd.read_csv(main_catalog_path)
        accepted_in_new = len(df_new_main[df_new_main['status'] == 'accepted'])
        print(f"✅ Verification: New main catalog has {accepted_in_new} accepted entries")
        
    except Exception as e:
        print(f"❌ Error updating catalog: {e}")
        return False
    
    print()
    print("=" * 80)
    print("🎉 MIGRATION COMPLETE!")
    print("=" * 80)
    print("Summary:")
    print(f"   • Moved {moved_count} accepted files to main folder")
    print(f"   • Updated main validation catalog")
    print(f"   • {accepted_in_new} accepted entries now in main catalog")
    if overwritten_count > 0:
        print(f"   • {overwritten_count} existing files were overwritten")
    print()
    print("Next steps:")
    print("   1. Verify the files are in the correct locations")
    print("   2. Re-run your original diagnostics to confirm consistency")
    print("   3. Consider cleaning up the copy folders if everything looks good")
    
    return True

# Run the migration
success = move_ling_files_to_main()

if success:
    print("\n" + "=" * 40)
    print("VERIFICATION CHECK")
    print("=" * 40)
    
    # Quick verification: reload the updated catalog and check consistency
    df_ling_updated = pd.read_csv("../data/conceptual-error-candidates/validation_catalog_ling.csv")
    df_ling_accepted = df_ling_updated[df_ling_updated['status'] == 'accepted']
    
    print(f"Updated Ling catalog has {len(df_ling_accepted)} accepted entries")
    
    # Check a few files to make sure they're in the right place
    sample_size = min(3, len(df_ling_accepted))
    print(f"Checking {sample_size} sample files...")
    
    for idx, (_, row) in enumerate(df_ling_accepted.head(sample_size).iterrows()):
        filepath = str(row['filepath']).replace('\\', '/')
        full_path = Path("..") / filepath
        exists = full_path.exists()
        status = "✅" if exists else "❌"
        print(f"   {status} Index {row['index']}: {exists}")

MOVING LING'S FILES TO MAIN FOLDERS
📋 Loading Ling's copy catalog...
Found 110 accepted entries to process

📁 Moving accepted files...
✅ Moving: tier4/5691/google_gemini-2.5-flash_incomplete_calculation_full_meal_cost_before_voucher_.json
✅ Moving: tier4/7184/google_gemini-2.5-flash_operator_swap_total_profits_.json
✅ Moving: tier4/7146/google_gemini-2.5-flash_operator_swap_ginger_tsp_.json
✅ Moving: tier4/2407/google_gemini-2.5-flash_operator_swap_wanda_treats_.json
✅ Moving: tier4/1111/google_gemini-2.5-flash_operator_swap_total_steak_pounds_.json
✅ Moving: tier4/6580/google_gemini-2.5-flash_incorrect_operand_cost_per_gallon_dollars_cost_1_cent.json
✅ Moving: tier4/3517/google_gemini-2.5-flash_operator_swap_sale_cost_dollars_.json
✅ Moving: tier4/2512/google_gemini-2.5-flash_incorrect_operand_produce_cost_eggplant_pounds.json
✅ Moving: tier4/7413/google_gemini-2.5-flash_operator_swap_total_profit_.json
✅ Moving: tier4/2844/google_gemini-2.5-flash_operator_swap_total_dolls_.json
✅ Mov

In [4]:
import pandas as pd
import json
from pathlib import Path
import shutil

def diagnose_null_erroneous_line_numbers():
    """
    Diagnose how many accepted files have null erroneous_line_number values for each reviewer.
    """
    print("=" * 80)
    print("DIAGNOSTIC: NULL ERRONEOUS LINE NUMBERS")
    print("=" * 80)
    
    # Load all validation catalogs
    validation_catalogs = {}
    parent_dir = "../data/conceptual-error-candidates/validation_catalog_"
    
    for reviewer in ['ali', 'arvind', 'ling', 'mauro', 'yewei']:
        try:
            df = pd.read_csv(f"{parent_dir}{reviewer}.csv")
            validation_catalogs[reviewer] = df[df['status'] == 'accepted']
        except FileNotFoundError:
            print(f"⚠️  Catalog not found for {reviewer}")
            validation_catalogs[reviewer] = pd.DataFrame()
    
    total_null_count = 0
    total_files_checked = 0
    
    for reviewer_name, df in validation_catalogs.items():
        if len(df) == 0:
            print(f"\n📋 REVIEWER: {reviewer_name.upper()}")
            print("-" * 50)
            print("No accepted files found.")
            continue
            
        print(f"\n📋 REVIEWER: {reviewer_name.upper()}")
        print("-" * 50)
        
        null_count = 0
        valid_count = 0
        error_count = 0
        
        for idx, row in df.iterrows():
            try:
                # Normalize the file path
                normalized_filepath = str(row['filepath']).replace('\\', '/')
                file_path = Path("..") / normalized_filepath
                
                if not file_path.exists():
                    error_count += 1
                    continue
                
                # Load and check the JSON
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                
                erroneous_line = json_data.get('error_details', {}).get('erroneous_line_number')
                
                if erroneous_line is None or erroneous_line == "null":
                    null_count += 1
                    # Show a few examples
                    if null_count <= 3:
                        print(f"   • Index {row['index']}: {row['mutation_type']} - NULL erroneous_line_number")
                else:
                    valid_count += 1
                    
            except Exception as e:
                error_count += 1
                print(f"   ❌ Error reading file for index {row['index']}: {e}")
        
        total_files_checked += len(df)
        total_null_count += null_count
        
        print(f"📊 SUMMARY:")
        print(f"   • Total accepted files: {len(df)}")
        print(f"   • Files with NULL erroneous_line_number: {null_count}")
        print(f"   • Files with valid erroneous_line_number: {valid_count}")
        print(f"   • Files with read errors: {error_count}")
        
        if len(df) > 0:
            null_percentage = (null_count / len(df)) * 100
            print(f"   • Percentage with NULL: {null_percentage:.1f}%")
    
    print("\n" + "=" * 80)
    print(f"🎯 OVERALL SUMMARY:")
    print(f"   • Total accepted files checked: {total_files_checked}")
    print(f"   • Total files with NULL erroneous_line_number: {total_null_count}")
    if total_files_checked > 0:
        overall_percentage = (total_null_count / total_files_checked) * 100
        print(f"   • Overall percentage with NULL: {overall_percentage:.1f}%")
    print("=" * 80)
    
    return total_null_count

In [5]:
null_count = diagnose_null_erroneous_line_numbers()

DIAGNOSTIC: NULL ERRONEOUS LINE NUMBERS

📋 REVIEWER: ALI
--------------------------------------------------
📊 SUMMARY:
   • Total accepted files: 341
   • Files with NULL erroneous_line_number: 0
   • Files with valid erroneous_line_number: 341
   • Files with read errors: 0
   • Percentage with NULL: 0.0%

📋 REVIEWER: ARVIND
--------------------------------------------------
📊 SUMMARY:
   • Total accepted files: 91
   • Files with NULL erroneous_line_number: 0
   • Files with valid erroneous_line_number: 91
   • Files with read errors: 0
   • Percentage with NULL: 0.0%

📋 REVIEWER: LING
--------------------------------------------------
📊 SUMMARY:
   • Total accepted files: 110
   • Files with NULL erroneous_line_number: 0
   • Files with valid erroneous_line_number: 110
   • Files with read errors: 0
   • Percentage with NULL: 0.0%

📋 REVIEWER: MAURO
--------------------------------------------------
📊 SUMMARY:
   • Total accepted files: 312
   • Files with NULL erroneous_line_number

In [12]:
df_shortlist = pd.read_csv("../data/conceptual-error-candidates/conceptual_candidates_shortlist.csv")

df_shortlist["erroneous_line_number"].value_counts()

erroneous_line_number
FA    538
L1    503
L2    369
L3    291
L4    156
L5     56
L6     16
L7      7
L8      5
L9      1
Name: count, dtype: int64

In [6]:
def repair_null_erroneous_line_numbers():
    """
    Repair null erroneous_line_number values using the conceptual_candidates_shortlist.csv file.
    """
    print("=" * 80)
    print("REPAIR: NULL ERRONEOUS LINE NUMBERS")
    print("=" * 80)
    
    # Load the shortlist CSV for reference
    shortlist_path = "../data/conceptual-error-candidates/conceptual_candidates_shortlist.csv"
    try:
        df_shortlist = pd.read_csv(shortlist_path)
        print(f"✅ Loaded shortlist with {len(df_shortlist)} entries")
    except FileNotFoundError:
        print(f"❌ Shortlist file not found: {shortlist_path}")
        return False
    
    # Create a lookup dictionary: (index, tier, mutation_type) -> erroneous_line_number
    lookup = {}
    for _, row in df_shortlist.iterrows():
        key = (row['index'], row['tier'], row['mutation_type'])
        lookup[key] = row['erroneous_line_number']
    
    print(f"✅ Created lookup table with {len(lookup)} entries")
    print()
    
    # Load all validation catalogs
    validation_catalogs = {}
    parent_dir = "../data/conceptual-error-candidates/validation_catalog_"
    
    for reviewer in ['ali', 'arvind', 'ling', 'mauro', 'yewei']:
        try:
            df = pd.read_csv(f"{parent_dir}{reviewer}.csv")
            validation_catalogs[reviewer] = df[df['status'] == 'accepted']
        except FileNotFoundError:
            validation_catalogs[reviewer] = pd.DataFrame()
    
    total_repaired = 0
    total_not_found = 0
    total_already_valid = 0
    
    for reviewer_name, df in validation_catalogs.items():
        if len(df) == 0:
            continue
            
        print(f"📋 REVIEWER: {reviewer_name.upper()}")
        print("-" * 50)
        
        repaired_count = 0
        not_found_count = 0
        already_valid_count = 0
        error_count = 0
        
        for idx, row in df.iterrows():
            try:
                # Normalize the file path
                normalized_filepath = str(row['filepath']).replace('\\', '/')
                file_path = Path("..") / normalized_filepath
                
                if not file_path.exists():
                    error_count += 1
                    continue
                
                # Load the JSON
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                
                current_erroneous_line = json_data.get('error_details', {}).get('erroneous_line_number')
                
                # Check if repair is needed
                if current_erroneous_line is not None and current_erroneous_line != "null":
                    already_valid_count += 1
                    continue
                
                # Look up the correct value
                key = (row['index'], row['tier'], row['mutation_type'])
                if key in lookup:
                    correct_erroneous_line = lookup[key]
                    
                    # Create backup
                    backup_path = file_path.with_suffix('.json.backup')
                    shutil.copy2(file_path, backup_path)
                    
                    # Update the JSON
                    json_data['error_details']['erroneous_line_number'] = correct_erroneous_line
                    
                    # Save the updated JSON
                    with open(file_path, 'w', encoding='utf-8') as f:
                        json.dump(json_data, f, indent=2, ensure_ascii=False)
                    
                    print(f"   ✅ Repaired Index {row['index']}: {current_erroneous_line} → {correct_erroneous_line}")
                    repaired_count += 1
                    
                else:
                    print(f"   ❌ Not found in shortlist: Index {row['index']}, Tier {row['tier']}, Type {row['mutation_type']}")
                    not_found_count += 1
                    
            except Exception as e:
                error_count += 1
                print(f"   ❌ Error processing index {row['index']}: {e}")
        
        total_repaired += repaired_count
        total_not_found += not_found_count
        total_already_valid += already_valid_count
        
        print(f"📊 SUMMARY:")
        print(f"   • Files repaired: {repaired_count}")
        print(f"   • Files already valid: {already_valid_count}")
        print(f"   • Files not found in shortlist: {not_found_count}")
        print(f"   • Files with errors: {error_count}")
        print()
    
    print("=" * 80)
    print(f"🎯 REPAIR SUMMARY:")
    print(f"   • Total files repaired: {total_repaired}")
    print(f"   • Total files already valid: {total_already_valid}")
    print(f"   • Total files not found in shortlist: {total_not_found}")
    print("=" * 80)
    
    if total_repaired > 0:
        print("✅ Repair completed! Backup files (.json.backup) were created for all modified files.")
    
    return total_repaired > 0

# Run diagnostic first
print("🔍 Running diagnostic...")
null_count = diagnose_null_erroneous_line_numbers()

if null_count > 0:
    print("\n" + "🔧 Running repair...")
    repair_success = repair_null_erroneous_line_numbers()
    
    if repair_success:
        print("\n" + "🔍 Running post-repair diagnostic...")
        diagnose_null_erroneous_line_numbers()
else:
    print("\n✅ No null erroneous_line_number values found. No repair needed!")

🔍 Running diagnostic...
DIAGNOSTIC: NULL ERRONEOUS LINE NUMBERS

📋 REVIEWER: ALI
--------------------------------------------------
   • Index 370: operator_swap - NULL erroneous_line_number
   • Index 2214: incorrect_operand - NULL erroneous_line_number
   • Index 389: incorrect_operand - NULL erroneous_line_number
📊 SUMMARY:
   • Total accepted files: 341
   • Files with NULL erroneous_line_number: 41
   • Files with valid erroneous_line_number: 300
   • Files with read errors: 0
   • Percentage with NULL: 12.0%

📋 REVIEWER: ARVIND
--------------------------------------------------
📊 SUMMARY:
   • Total accepted files: 91
   • Files with NULL erroneous_line_number: 0
   • Files with valid erroneous_line_number: 91
   • Files with read errors: 0
   • Percentage with NULL: 0.0%

📋 REVIEWER: LING
--------------------------------------------------
📊 SUMMARY:
   • Total accepted files: 110
   • Files with NULL erroneous_line_number: 0
   • Files with valid erroneous_line_number: 110
   • 